# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# Obtain Data

Read csv with migration data into pandas

In [2]:
migration_df = pd.read_csv('Data/migration_data/migration_data.csv', header=1)
migration_df.head()

,id,Geographic Area Name,Estimate!!Total!!Population 1 year and over,Margin of Error!!Total!!Population 1 year and over,Estimate!!Total!!Population 1 year and over!!AGE!!1 to 4 years,Margin of Error!!Total!!Population 1 year and over!!AGE!!1 to 4 years,Estimate!!Total!!Population 1 year and over!!AGE!!5 to 17 years,Margin of Error!!Total!!Population 1 year and over!!AGE!!5 to 17 years,Estimate!!Total!!Population 1 year and over!!AGE!!18 to 24 years,Margin of Error!!Total!!Population 1 year and over!!AGE!!18 to 24 years,...,Estimate!!Moved; from abroad!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population 1 year and over for whom poverty status is determined!!At or above 150 percent of the poverty level,Margin of Error!!Moved; from abroad!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population 1 year and over for whom poverty status is determined!!At or above 150 percent of the poverty level,Estimate!!Moved; from abroad!!HOUSING TENURE!!Population 1 year and over in housing units,Margin of Error!!Moved; from abroad!!HOUSING TENURE!!Population 1 year and over in housing units,Estimate!!Moved; from abroad!!HOUSING TENURE!!Population 1 year and over in housing units!!Householder lived in owner-occupied housing units,Margin of Error!!Moved; from abroad!!HOUSING TENURE!!Population 1 year and over in housing units!!Householder lived in owner-occupied housing units,Estimate!!Moved; from abroad!!HOUSING TENURE!!Population 1 year and over in housing units!!Householder lived in renter-occupied housing units,Margin of Error!!Moved; from abroad!!HOUSING TENURE!!Population 1 year and over in housing units!!Householder lived in renter-occupied housing units,Estimate!!Moved; from abroad!!PERCENT ALLOCATED!!Residence 1 year ago,Margin of Error!!Moved; from abroad!!PERCENT ALLOCATED!!Residence 1 year ago
0,0500000US01003,"Baldwin County, Alabama",221737,907,9119,1103,36287,926,16614,1496,...,0.1,0.1,0.2,0.2,0.1,0.1,0.7,1.1,(X),(X)
1,0500000US01015,"Calhoun County, Alabama",112208,658,5302,657,17775,200,10047,750,...,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.5,(X),(X)
2,0500000US01043,"Cullman County, Alabama",83222,346,4764,706,13503,609,6160,397,...,0.0,0.3,0.0,0.2,0.0,0.3,0.0,0.9,(X),(X)
3,0500000US01049,"DeKalb County, Alabama",N,N,N,N,N,N,N,N,...,N,N,0.1,0.1,0.1,0.1,0.1,0.1,(X),(X)
4,0500000US01051,"Elmore County, Alabama",N,N,N,N,N,N,N,N,...,N,N,0.9,1.2,0.0,0.3,4.4,5.6,(X),(X)


Drop unneeded columns from migration_df, scrub formatting and combine interstate and intrastate migration columns into migration column

In [3]:
migration_df = migration_df[['Geographic Area Name',
                                'Estimate!!Moved; from different county, same state!!Population 1 year and over',
                                'Estimate!!Moved; from different  state!!Population 1 year and over']]
migration_df.replace({'N': np.nan}, inplace=True)
migration_df.columns = ['County', 'Intrastate Migration %', 'Interstate Migration %']
migration_df = migration_df.astype({'County': object, 'Intrastate Migration %': float,
                                    'Interstate Migration %': float})
migration_df['Inmigration %'] =  migration_df['Intrastate Migration %'] + migration_df['Interstate Migration %']
migration_df.head()

,County,Intrastate Migration %,Interstate Migration %,Inmigration %
0,"Baldwin County, Alabama",2.0,2.4,4.4
1,"Calhoun County, Alabama",2.7,4.1,6.8
2,"Cullman County, Alabama",3.5,0.9,4.4
3,"DeKalb County, Alabama",NaN,NaN,NaN
4,"Elmore County, Alabama",NaN,NaN,NaN


Read csv with economic data into pandas

In [4]:
economic_df = pd.read_csv('Data/economic_data/economic_data.csv', header=1)
economic_df.head()

,id,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,Percent!!EMPLOYMENT STATUS!!Population 16 years and over,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Percent!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force,...,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!65 years and over,Percent Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!18 years and over!!65 years and over,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!People in families,Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!People in families,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!People in families,Percent Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!People in families,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Unrelated individuals 15 years and over,Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Unrelated individuals 15 years and over,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Unrelated individuals 15 years and over,Percent Margin of Error!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All people!!Unrelated individuals 15 years and over
0,0500000US01003,"Baldwin County, Alabama",183875,1197,183875,(X),101561,4760,55.2,2.6,...,9.1,3.1,(X),(X),8.3,2.5,(X),(X),18.5,4.1
1,0500000US01015,"Calhoun County, Alabama",92308,597,92308,(X),51001,2668,55.3,2.9,...,7.2,2.7,(X),(X),12.8,3.4,(X),(X),35.0,7.3
2,0500000US01043,"Cullman County, Alabama",66939,491,66939,(X),38222,1793,57.1,2.6,...,6.7,2.8,(X),(X),7.9,3.0,(X),(X),24.7,6.2
3,0500000US01049,"DeKalb County, Alabama",56471,671,56471,(X),34492,1971,61.1,3.4,...,8.5,3.6,(X),(X),17.9,5.5,(X),(X),28.6,5.9
4,0500000US01051,"Elmore County, Alabama",65674,847,65674,(X),37693,2523,57.4,3.9,...,5.1,3.1,(X),(X),10.5,4.0,(X),(X),18.2,7.0


Drop unneeded columns from economic_df, calculate unemployment rate and scrub formatting

In [5]:
economic_df = economic_df[['Geographic Area Name',
                           'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force',
                           'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed',
                           'Estimate!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Per capita income (dollars)']]
economic_df.columns = ['County', 'Labor Force', 'Employment', 'Per Capita Income']
economic_df = economic_df.astype({'County': object, 'Labor Force': int, 'Employment': int, 
                                  'Per Capita Income': int})
economic_df['Unemployment Rate'] = economic_df['Labor Force'] - economic_df['Employment']
economic_df['Unemployment Rate'] = economic_df['Unemployment Rate'] / economic_df['Labor Force']
economic_df.head()

,County,Labor Force,Employment,Per Capita Income,Unemployment Rate
0,"Baldwin County, Alabama",101561,96012,32443,0.054637
1,"Calhoun County, Alabama",51001,45641,24579,0.105096
2,"Cullman County, Alabama",38222,36439,23968,0.046649
3,"DeKalb County, Alabama",34492,32544,21939,0.056477
4,"Elmore County, Alabama",37693,35358,31396,0.061948


Read csv with housing data into pandas

In [6]:
housing_df = pd.read_csv('Data/housing_data/housing_data.csv', header=1)
housing_df.head()

,id,Geographic Area Name,Estimate!!HOUSING OCCUPANCY!!Total housing units,Margin of Error!!HOUSING OCCUPANCY!!Total housing units,Percent!!HOUSING OCCUPANCY!!Total housing units,Percent Margin of Error!!HOUSING OCCUPANCY!!Total housing units,Estimate!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units,Margin of Error!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units,Percent!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units,Percent Margin of Error!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units,...,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30.0 to 34.9 percent,Percent Margin of Error!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30.0 to 34.9 percent,Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!35.0 percent or more,Margin of Error!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!35.0 percent or more,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!35.0 percent or more,Percent Margin of Error!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!35.0 percent or more,Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Not computed,Margin of Error!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Not computed,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Not computed,Percent Margin of Error!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Not computed
0,0500000US01003,"Baldwin County, Alabama",119425,483,119425,(X),82325,3231,68.9,2.7,...,8.2,4.5,6003,1790,41.7,10.0,2648,1235,(X),(X)
1,0500000US01015,"Calhoun County, Alabama",53809,404,53809,(X),44636,2010,83.0,3.7,...,8.4,5.2,4659,1199,38.6,8.6,1074,642,(X),(X)
2,0500000US01043,"Cullman County, Alabama",38256,332,38256,(X),32090,1233,83.9,3.2,...,8.0,5.7,1839,569,25.5,7.6,575,279,(X),(X)
3,0500000US01049,"DeKalb County, Alabama",31606,256,31606,(X),24880,1347,78.7,4.1,...,22.6,10.7,1292,711,22.7,10.4,1392,635,(X),(X)
4,0500000US01051,"Elmore County, Alabama",34481,471,34481,(X),30712,1330,89.1,3.4,...,4.3,3.6,3726,1274,54.1,11.8,968,629,(X),(X)


Drop unneeded columns from housing_df and scrub formatting

In [7]:
housing_df = housing_df[['Geographic Area Name', 'Estimate!!VALUE!!Owner-occupied units!!Median (dollars)']]
housing_df.columns = ['County', 'Median House Price']
housing_df = housing_df.astype({'County': object, 'Median House Price': int})
housing_df.head()

,County,Median House Price
0,"Baldwin County, Alabama",223100
1,"Calhoun County, Alabama",121600
2,"Cullman County, Alabama",137600
3,"DeKalb County, Alabama",115200
4,"Elmore County, Alabama",176100


Read csv with education data into pandas

In [8]:
education_df = pd.read_csv('Data/education_data/education_data.csv', header=1)
education_df.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,...,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Less than high school graduate,Margin of Error!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Less than high school graduate,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency),Margin of Error!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency),Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Some college or associate's degree,Margin of Error!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Some college or associate's degree,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Bachelor's degree,Margin of Error!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Bachelor's degree,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Graduate or professional degree,Margin of Error!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Graduate or professional degree
0,0500000US01003,"Baldwin County, Alabama",16614,1496,2316,1329,7398,2126,5354,1439,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,0500000US01015,"Calhoun County, Alabama",10047,750,897,568,2099,981,6507,842,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,0500000US01043,"Cullman County, Alabama",6160,397,731,375,1871,552,3111,624,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,0500000US01049,"DeKalb County, Alabama",7072,650,1240,731,2441,788,3122,953,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,0500000US01051,"Elmore County, Alabama",5605,681,1053,549,1928,765,2179,567,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


Drop unneeded columns from education_df, calculate bachleros degree proportion and scrub formatting

In [9]:
education_df = education_df[['Geographic Area Name', 
                             'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over',
                             "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree"]]
education_df.columns = ['County', 'Population 25+', 'Bachelors Degree 25+']
education_df = education_df.astype({'County': object, 'Population 25+': int, 'Bachelors Degree 25+': int})
education_df['Bachelors Degree %'] = education_df['Bachelors Degree 25+']/education_df['Population 25+']
education_df.head()

,County,Population 25+,Bachelors Degree 25+,Bachelors Degree %
0,"Baldwin County, Alabama",159717,32781,0.205244
1,"Calhoun County, Alabama",79084,9082,0.114840
2,"Cullman County, Alabama",58795,5935,0.100944
3,"DeKalb County, Alabama",47007,3693,0.078563
4,"Elmore County, Alabama",57553,9418,0.163640


Read csv with commute data into pandas

In [10]:
commute_df = pd.read_csv('Data/commute_data/commute_data.csv', header=1)
commute_df.head()

,id,Geographic Area Name,Estimate!!Total!!Workers 16 years and over,Margin of Error!!Total!!Workers 16 years and over,"Estimate!!Total!!Workers 16 years and over!!MEANS OF TRANSPORTATION TO WORK!!Car, truck, or van","Margin of Error!!Total!!Workers 16 years and over!!MEANS OF TRANSPORTATION TO WORK!!Car, truck, or van","Estimate!!Total!!Workers 16 years and over!!MEANS OF TRANSPORTATION TO WORK!!Car, truck, or van!!Drove alone","Margin of Error!!Total!!Workers 16 years and over!!MEANS OF TRANSPORTATION TO WORK!!Car, truck, or van!!Drove alone","Estimate!!Total!!Workers 16 years and over!!MEANS OF TRANSPORTATION TO WORK!!Car, truck, or van!!Carpooled","Margin of Error!!Total!!Workers 16 years and over!!MEANS OF TRANSPORTATION TO WORK!!Car, truck, or van!!Carpooled",...,Estimate!!Female!!PERCENT ALLOCATED!!Private vehicle occupancy,Margin of Error!!Female!!PERCENT ALLOCATED!!Private vehicle occupancy,Estimate!!Female!!PERCENT ALLOCATED!!Place of work,Margin of Error!!Female!!PERCENT ALLOCATED!!Place of work,Estimate!!Female!!PERCENT ALLOCATED!!Time of departure to go to work,Margin of Error!!Female!!PERCENT ALLOCATED!!Time of departure to go to work,Estimate!!Female!!PERCENT ALLOCATED!!Travel time to work,Margin of Error!!Female!!PERCENT ALLOCATED!!Travel time to work,Estimate!!Female!!PERCENT ALLOCATED!!Vehicles available,Margin of Error!!Female!!PERCENT ALLOCATED!!Vehicles available
0,0500000US01003,"Baldwin County, Alabama",94867,4720,94.3,1.6,85.6,2.5,8.7,2.0,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,0500000US01015,"Calhoun County, Alabama",N,N,N,N,N,N,N,N,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,0500000US01043,"Cullman County, Alabama",35840,1743,96.5,1.6,83.6,4.0,12.9,3.9,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,0500000US01049,"DeKalb County, Alabama",N,N,N,N,N,N,N,N,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,0500000US01051,"Elmore County, Alabama",N,N,N,N,N,N,N,N,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


Drop unneeded columns from commute_df and scrub data

In [11]:
commute_df = commute_df[['Geographic Area Name', 
                         'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!Mean travel time to work (minutes)']]
commute_df.columns = ['County', 'Commute Time']
commute_df = commute_df.astype({'County': object, 'Commute Time': float})
commute_df.head()

,County,Commute Time
0,"Baldwin County, Alabama",26.1
1,"Calhoun County, Alabama",22.3
2,"Cullman County, Alabama",27.9
3,"DeKalb County, Alabama",24.9
4,"Elmore County, Alabama",26.5


Read csv with weather data into pandas

In [12]:
weather_df = pd.read_csv('Data/weather_data/weather_data.csv', header=3)
weather_df.head()

,Location ID,Location,Value,Rank,Anomaly (1901-2000 base period),1901-2000 Mean
0,AL-001,Autauga County,65.4,114,1.5,63.9
1,AL-003,Baldwin County,67.4,92,0.6,66.8
2,AL-005,Barbour County,65.3,86,0.5,64.8
3,AL-007,Bibb County,63.3,83,0.6,62.7
4,AL-009,Blount County,62.0,109,1.4,60.6


Drop unneeded columns from weather_df and scrub formatting

In [13]:
weather_df = weather_df[['Location ID', 'Location', '1901-2000 Mean']]
weather_df.head()
weather_df.columns = ['State', 'County', 'Mean Temperature']
weather_df.head()

,State,County,Mean Temperature
0,AL-001,Autauga County,63.9
1,AL-003,Baldwin County,66.8
2,AL-005,Barbour County,64.8
3,AL-007,Bibb County,62.7
4,AL-009,Blount County,60.6


Reformat county names to match other tables

In [14]:
state_abbreviations_df = pd.read_csv('Data/weather_data/state_abbreviations.csv')
state_abbreviations_df.head()

def state_column(row):
    abbreviation = row.split('-')[0]
    return state_abbreviations_df[state_abbreviations_df['Abbreviation']==abbreviation]['State'].values[0]
weather_df['State'] = weather_df['State'].apply(state_column)
weather_df['County'] = weather_df['County'] + ', ' + weather_df['State']
weather_df.drop(columns='State', inplace=True)
weather_df.head()

,County,Mean Temperature
0,"Autauga County, Alabama",63.9
1,"Baldwin County, Alabama",66.8
2,"Barbour County, Alabama",64.8
3,"Bibb County, Alabama",62.7
4,"Blount County, Alabama",60.6


Merge data from dfs with census data into a single df

In [15]:
df_list = [migration_df, economic_df, housing_df, education_df, commute_df]
df = migration_df['County']
for data_df in df_list:
    df = pd.merge(df, data_df, how='inner', on='County')
print(df.shape)
df.head()

(840, 13)


,County,Intrastate Migration %,Interstate Migration %,Inmigration %,Labor Force,Employment,Per Capita Income,Unemployment Rate,Median House Price,Population 25+,Bachelors Degree 25+,Bachelors Degree %,Commute Time
0,"Baldwin County, Alabama",2.0,2.4,4.4,101561,96012,32443,0.054637,223100,159717,32781,0.205244,26.1
1,"Calhoun County, Alabama",2.7,4.1,6.8,51001,45641,24579,0.105096,121600,79084,9082,0.114840,22.3
2,"Cullman County, Alabama",3.5,0.9,4.4,38222,36439,23968,0.046649,137600,58795,5935,0.100944,27.9
3,"DeKalb County, Alabama",NaN,NaN,NaN,34492,32544,21939,0.056477,115200,47007,3693,0.078563,24.9
4,"Elmore County, Alabama",NaN,NaN,NaN,37693,35358,31396,0.061948,176100,57553,9418,0.163640,26.5


Reformat df county names to match weather_df county names and merge the two dfs

In [16]:
df['County'] = df['County'].apply(lambda x: x.replace('city,', 'City,'))
df['County'] = df['County'].apply(lambda x: x.replace('ñ', 'n'))
weather_df['County'] = weather_df['County'].apply(lambda x: x.replace('Washington, D.C., Maryland',
                                                                      'District of Columbia, District of Columbia'))

df = pd.merge(df, weather_df, how='outer', on='County')
print(df.shape)
df.head()

(3151, 14)


,County,Intrastate Migration %,Interstate Migration %,Inmigration %,Labor Force,Employment,Per Capita Income,Unemployment Rate,Median House Price,Population 25+,Bachelors Degree 25+,Bachelors Degree %,Commute Time,Mean Temperature
0,"Baldwin County, Alabama",2.0,2.4,4.4,101561.0,96012.0,32443.0,0.054637,223100.0,159717.0,32781.0,0.205244,26.1,66.8
1,"Calhoun County, Alabama",2.7,4.1,6.8,51001.0,45641.0,24579.0,0.105096,121600.0,79084.0,9082.0,0.114840,22.3,60.9
2,"Cullman County, Alabama",3.5,0.9,4.4,38222.0,36439.0,23968.0,0.046649,137600.0,58795.0,5935.0,0.100944,27.9,60.3
3,"DeKalb County, Alabama",NaN,NaN,NaN,34492.0,32544.0,21939.0,0.056477,115200.0,47007.0,3693.0,0.078563,24.9,59.3
4,"Elmore County, Alabama",NaN,NaN,NaN,37693.0,35358.0,31396.0,0.061948,176100.0,57553.0,9418.0,0.163640,26.5,63.8


# Scrub data